In [8]:
# -*- coding: utf-8 -*-
import os
import json
import re
import string
import pandas as pd
import nltk
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.tokenize import MWETokenizer
from sklearn.decomposition import TruncatedSVD
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import ast 

np.random.seed(222)

In [9]:
def get_cosine_sim(*strs): 
    vectors = [t for t in get_vectors(*strs)]
    return cosine_similarity(vectors)
    
def get_vectors(*strs):
    text = [t for t in strs]
    vectorizer = CountVectorizer(text)
    vectorizer.fit(text)
    return vectorizer.transform(text).toarray()

In [10]:
text = pd.read_csv("../CSV_Data/preprocessed_data.csv")

In [11]:
type(text.iloc[0][' body'])

str

In [12]:
text[' body'] = text[' body'].apply(ast.literal_eval)

In [85]:
temp_frame = pd.DataFrame()
for i in range(len(text)):
    if text.iloc[i]["Issue"]==1.0:
        temp_frame = temp_frame.append(text.iloc[i], ignore_index=True)
        
print(len(temp_frame))
temp_frame.head()

547


,body,time,Doc_num,Issue,Unnamed: 0,index,title
0,"[north, korea, appear, achieve, significant, l...",2015-01-06 14:25:00,23687.0,1.0,23687.0,2688.0,"['korea', 'significant', 'technology', 'miniat..."
1,"[north, korea, capability, miniaturize, nuclea...",2015-01-06 21:30:00,23673.0,1.0,23673.0,2674.0,"['seoul', 'report', 'progress', 'nuke', 'capab..."
2,"[unite, state, evidence, north, korea, master,...",2015-01-09 09:56:00,23625.0,1.0,23625.0,2626.0,"['u.s.', 'evidence', 'show', 'master', 'warhea..."
3,"[north, korea, appear, equip, submarine, make,...",2015-01-09 09:58:00,23624.0,1.0,23624.0,2625.0,"['korea', 'make', 'submarine', 'capable', 'fir..."
4,"[recent, satellite, image, offer, fresh, evide...",2015-01-09 12:40:00,23618.0,1.0,23618.0,2619.0,"['korea', 'develop', 'base', 'missiles', 'thin..."


In [86]:
text[' time'] = pd.to_datetime(text[' time'])
text = text.sort_values(by=' time', ascending=True)

In [87]:
temp_frame.columns

Index([' body', ' time', 'Doc_num', 'Issue', 'Unnamed: 0', 'index', 'title'], dtype='object')

In [88]:
temp_frame[' body']

0      [north, korea, appear, achieve, significant, l...
1      [north, korea, capability, miniaturize, nuclea...
2      [unite, state, evidence, north, korea, master,...
3      [north, korea, appear, equip, submarine, make,...
4      [recent, satellite, image, offer, fresh, evide...
                             ...                        
542    [south, korea, defense, ministry, friday, nort...
543    [north, korea, appear, advance, submarine, lau...
544    [north, korea, latest, missile, test, u.s., ex...
545    [north, korea, likely, seek, dialogue, unite, ...
546    [washington, satellite, imagery, show, little,...
Name:  body, Length: 547, dtype: object

In [89]:
temp_frame[' time'] = pd.to_datetime(temp_frame[' time']).dt.date
temp_frame = temp_frame.sort_values([' time'])
len(temp_frame)

547

In [90]:
print(temp_frame.iloc[0][' time'])
print(temp_frame.iloc[-1][' time'])

2015-01-06
2017-12-22


# Make event list

In [91]:
temp_frame.columns

Index([' body', ' time', 'Doc_num', 'Issue', 'Unnamed: 0', 'index', 'title'], dtype='object')

In [92]:
def make_event_list(df, similarity):
    event_list = []
    visit = [0 for _ in range(len(df))]
    for i in range(len(df)-1):
        add = False
        if visit[i] == 0:    
            start_time = df.iloc[i][' time']
            until_7day_list = []
            until_7day_list.append((i, " ".join(df.iloc[i][' body'])))
            for j in range(i+1, len(df)):
                end_time = df.iloc[j][' time']

                if (end_time - start_time).days <= 7:
                    until_7day_list.append((j," ".join(df.iloc[j][' body'])))
                else:
                    break

            on_event_issue_set = set()
            on_event_issue_set.add(i)
            for k in range(1,len(until_7day_list)):
                if visit[until_7day_list[k][0]] == 1:
                    continue    

                for index in on_event_issue_set:
                    all_above = True
                    if get_cosine_sim(" ".join(df.iloc[index][' body']), until_7day_list[k][1])[0][1] < similarity:
                        all_above = False
                        break

                if all_above:    
                    on_event_issue_set.add(until_7day_list[k][0])
                    visit[until_7day_list[k][0]] = 1
                    add = True

        if add:
            visit[i] = 1
            event_list.append(on_event_issue_set)
    return event_list

In [93]:
event_list = make_event_list(temp_frame, 0.5)

In [94]:
len(event_list)

119

In [104]:
event_list

[{0, 1, 4},
 {3, 5},
 {15, 16},
 {17, 18},
 {19, 20, 21, 24, 25, 26},
 {22, 23, 27},
 {29, 30},
 {33, 34, 35},
 {37, 38, 41},
 {40, 44},
 {47, 49},
 {48, 50, 51},
 {53, 54},
 {55, 56},
 {57, 58, 60, 61},
 {62, 64, 66},
 {65, 67, 68},
 {70, 71},
 {73, 74},
 {75, 76, 77},
 {82, 84, 88, 89},
 {91, 93, 94, 95, 98, 99, 104, 106},
 {97, 103, 105, 111, 113, 114, 117, 118, 122},
 {100, 107, 110, 115, 125, 126},
 {102, 108},
 {112, 123},
 {120, 121},
 {124, 128},
 {129, 130},
 {138, 139, 142, 143},
 {140, 146},
 {144, 157},
 {145, 147, 150, 151},
 {152, 154, 158, 162},
 {155, 156, 163},
 {159, 160, 161, 168},
 {164, 169},
 {165, 171},
 {166, 167, 176},
 {178, 179, 180, 181, 184, 186},
 {182, 183, 185, 187},
 {189, 191, 194, 195},
 {192, 193, 197, 201, 202, 203},
 {198, 204},
 {205, 206},
 {210, 211, 213},
 {212, 214, 215},
 {218, 219},
 {220, 221, 224, 226, 230, 234, 235},
 {222, 223, 225, 228, 229, 231, 233, 237},
 {227, 236},
 {232, 238},
 {241, 242, 243},
 {244, 245, 247},
 {248, 251, 252},


In [96]:
elem = event_list[0]
for e in sorted(elem):
    print(e)
    print(temp_frame.iloc[e]['title'])
    print(temp_frame.iloc[e][' time'])
print()

0
['korea', 'significant', 'technology', 'miniaturize', 'nuke', 'seoul']
2015-01-06
1
['seoul', 'report', 'progress', 'nuke', 'capability']
2015-01-06
4
['korea', 'develop', 'base', 'missiles', 'think', 'tank']
2015-01-09



In [97]:
def concat(*str):
    return " ".join(str)

# Make on-issue event list

In [98]:
def make_onissue_event_list(df, event_list, similarity):
    set_list = []
    set_visit = [0 for _ in range(len(event_list))]
    for i in range(len(event_list)-1):
        if set_visit[i]==1:
            continue
        target_set = event_list[i]
        target_sentence = ""
        for index in target_set:
            target_sentence =concat(target_sentence, str(df.iloc[index][' body']))

        target_sentence = target_sentence[1:]

        concat_list = []
        concat_index_list = []
        temp_target_set = target_set
        for j in range(i+1, len(event_list)):
            if set_visit[j]==1:
                continue
            compare_set = event_list[j]
            compare_sentence = ""
            for index in compare_set:
                compare_sentence = concat(compare_sentence, str(df.iloc[index][' body']))
            compare_sentence = compare_sentence[1:]

            if get_cosine_sim(target_sentence, compare_sentence)[0][1] >= similarity:

                if max(df.iloc[list(target_set)][' time']) >= min(df.iloc[list(compare_set)][' time']):
                    target_sentence = concat(target_sentence, " ".join(compare_sentence))
                    temp_target_set = temp_target_set.union(compare_set)
                    if not concat_index_list:
                        concat_index_list.append(i)
                    concat_index_list.append(j)
                else:
                    if not temp_target_set in concat_list:
                        concat_list.append(temp_target_set)
                    concat_list.append(compare_set)
                    concat_index_list.append(j)
                    for index in concat_index_list:                    
                        set_visit[index] = 1
        if concat_list:
            set_list.append(concat_list)
    return set_list

In [99]:
onissue_event_list = make_onissue_event_list(temp_frame, event_list, 0.7)

In [100]:
len(onissue_event_list)

17

In [101]:
len(onissue_event_list[0])

13

In [102]:
onissue_event_list

[[{0, 1, 4},
  {22, 23, 27},
  {62, 64, 66},
  {138, 139, 142, 143},
  {145, 147, 150, 151},
  {178, 179, 180, 181, 184, 186},
  {220, 221, 224, 226, 230, 234, 235},
  {316, 317},
  {482, 485},
  {489, 491, 500},
  {490, 494, 498, 499, 503, 505},
  {507, 516},
  {517, 518, 519, 520, 521, 522}],
 [{3, 5},
  {19, 20, 21, 24, 25, 26},
  {189, 191, 194, 195},
  {261, 262, 264, 265, 266, 267},
  {269, 270, 274}],
 [{29, 30},
  {65, 67, 68},
  {73, 74},
  {75, 76, 77},
  {82, 84, 88, 89},
  {192, 193, 197, 201, 202, 203},
  {222, 223, 225, 228, 229, 231, 233, 237},
  {241, 242, 243},
  {275, 277, 278},
  {318, 319, 321, 322, 323},
  {329, 331, 332, 333, 335, 336, 338, 339, 340, 341, 346},
  {352, 353, 354, 355, 357, 358},
  {356, 359},
  {391, 392, 393, 394, 402, 407},
  {395, 396, 408, 411, 413},
  {397, 398, 400},
  {417, 418, 421, 422},
  {442, 443, 444, 448, 453},
  {464, 466, 468, 470},
  {474, 475, 476},
  {530, 531, 533, 534, 535, 540}],
 [{33, 34, 35},
  {55, 56},
  {244, 245, 247},


# Check

In [128]:
def choose_representative(index_list):
    
    max_score = 0
    max_score_sentence_index = -1
    body_list = []
    
    for index in index_list:
        body_list.append((index, " ".join(temp_frame.iloc[index][' body'])))
    for index_n_body in body_list:
        index, body = index_n_body
        score = 0
        for index_n_comp_body in body_list:
            comp_index, comp_body = index_n_comp_body
            if index!=comp_index:
                cosine_sim = get_cosine_sim(body, comp_body)[0][1]
                score += cosine_sim
        score /= len(body_list) - 1
        if score > max_score:
            max_score = score
            max_score_sentence_index = index
    return max_score_sentence_index, temp_frame.iloc[max_score_sentence_index]['title']

In [129]:
for j, elem in enumerate(onissue_event_list):
    if len(elem)>=5:
        if j!=0:
            print("********************************************************************************")
        for i, ele in enumerate(elem):
            ele = sorted(ele)
            print(ele)
            print(choose_representative(ele))
#             for e in ele:
                # print(temp_frame.iloc[e]['title'] , " ,", temp_frame.iloc[e][' time'])
                
            if i!=len(elem)-1:
                print()
                print(" |")
                print(" V")
                print()

[0, 1, 4]
(0, "['korea', 'significant', 'technology', 'miniaturize', 'nuke', 'seoul']")

 |
 V

[22, 23, 27]
(22, "['submarine', 'missile', 'serious', 'threat']")

 |
 V

[62, 64, 66]
(62, "['u.s.', 'commander', 'korea', 'capable', 'fire', 'nuclear', 'missile', 'homeland']")

 |
 V

[138, 139, 142, 143]
(143, "['u.s.', 'downplay', 'nuclear', 'missile', 'capabilities', 'prepare', 'worst', 'scenario']")

 |
 V

[145, 147, 150, 151]
(145, "['korea', 'fire', 'ballistic', 'missiles', 'blow', 'flight']")

 |
 V

[178, 179, 180, 181, 184, 186]
(179, "['korea', 'show', 'sign', 'mobile', 'ballistic', 'missile', 'launch']")

 |
 V

[220, 221, 224, 226, 230, 234, 235]
(221, "['korea', 'fire', 'musudan', 'irbm', 'missiles']")

 |
 V

[316, 317]
(316, "['north', 'korea', '50kg', 'weapons', 'grade', 'plutonium', 'seoul', 'defense', 'white', 'paper']")

 |
 V

[482, 485]
(482, "['moon', 'urge', 'complete', 'overhaul', 'military']")

 |
 V

[489, 491, 500]
(500, "['chronology', 'north', 'korea', 'miss